In [27]:
import requests
from bs4 import BeautifulSoup
import csv

In [28]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

In [29]:
import pandas as pd

In [88]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver.wait = WebDriverWait(driver, 1000)
driver.get('https://bananarepublic.gap.com/browse/category.do?cid=69883&mlink=5002,16447437,w_dp_cn_dresses&clink=16447437')

In [5]:
driver.execute_script("window.scrollTo(20000, 30000);")

In [24]:
main_page = driver.page_source
main_soup = BeautifulSoup(main_page, "lxml")

In [52]:
def scrape(prod_pages,prod_names,prod_prices,prod_images,prod_desc,main_soup):
    for item in main_soup.find_all('div', {'class': 'product-card'}):
        url = item.find('a', {'class': 'product-card__link'})
        if url:
            url = 'https://bananarepublic.gap.com/'+url['href']
        else:
            url = None
            description = None
        prod_pages.append(url)
        # prod_desc.append(description)
        img = item.find('div',{'class':'product-card__image-wrapper'})
        if img:
            img_src = img.img['src']
            name = img.img['alt']
        else:
            img_src = None
            name = None
        prod_images.append(img_src)
        prod_names.append(name)
        price = item.find('div',{'class':'product-card-price'})
        if price:
            prod_price = price.div.span.get_text()
        else:
            prod_price = None
        prod_prices.append(prod_price)
    return prod_pages,prod_names,prod_prices,prod_images,prod_desc

In [53]:
prod_pages = []
prod_names = []
prod_prices = []
prod_dp = []
prod_images = []
prod_desc=[]
for i in range(100):
    driver.execute_script("window.scrollTo("+str(i*500)+", "+str((i+1)*500)+");")
    main_page = driver.page_source
    main_soup = BeautifulSoup(main_page, "lxml")
    prod_pages,prod_names,prod_prices,prod_images,prod_desc = scrape(prod_pages,prod_names,prod_prices,prod_images,prod_desc,main_soup)

In [54]:
len(prod_pages),len(prod_images),len(prod_names),len(prod_prices),len(prod_desc)

(30000, 30000, 30000, 30000, 0)

In [56]:
d = { 'image_link': prod_images,
         'product_name': prod_names, 
        'link': prod_pages, 
         'price': prod_prices}
#,
#    'description':prod_desc}
    
df = pd.DataFrame(d)
df['domain'] = 'Banana Republic'

In [57]:
df = df.dropna().drop_duplicates()

In [58]:
len(df)

300

In [309]:
df = df[['image_link','product_name','link','price','domain']]

In [60]:
for url in df.link:
    driver.get(url)
    desc = driver.page_source
    desc_soup = BeautifulSoup(desc, "lxml")
    for d in desc_soup.find('li',{'class':'product-information-item__list-item'}):
        description = d.get_text()
    prod_desc.append(description)

In [63]:
df['description']=prod_desc

In [66]:
df['price']=df.price.map(lambda x: float(x[1:]))
df

,image_link,product_name,link,price,domain,description
0,https://www3.assets-gap.com/webcontent/0016/73...,Chevron Pleated Midi Dress,https://bananarepublic.gap.com//browse/product...,149.0,Banana Republic,A flattering fit-and-flare style with oh-so-no...
1,https://www1.assets-gap.com/webcontent/0016/78...,Eyelet Pinafore Dress,https://bananarepublic.gap.com//browse/product...,159.0,Banana Republic,"Inspired by the romance of the prairie, this m..."
2,https://www2.assets-gap.com/webcontent/0016/64...,Paneled Fit-and-Flare Dress,https://bananarepublic.gap.com//browse/product...,119.0,Banana Republic,"Expert tailoring and a flirty, A-line skirt ma..."
3,https://www3.assets-gap.com/webcontent/0016/75...,Floral Drop-Waist Shift Dress,https://bananarepublic.gap.com//browse/product...,129.0,Banana Republic,"A classic shift dress in a modern, floral prin..."
4,https://www1.assets-gap.com/webcontent/0016/74...,Plaid Bi-Stretch Racer-Neck Sheath Dress,https://bananarepublic.gap.com//browse/product...,129.0,Banana Republic,Expert tailoring and a racer neck give this bo...
5,https://www1.assets-gap.com/webcontent/0016/64...,Sandwash Modal Knit Tank Dress,https://bananarepublic.gap.com//browse/product...,119.0,Banana Republic,"A column-style tank dress, made in a soft, kni..."
6,https://www1.assets-gap.com/webcontent/0016/57...,Floral Ruffle Wrap Dress,https://bananarepublic.gap.com//browse/product...,129.0,Banana Republic,A cascade of ruffles add a playful edge to thi...
7,https://www2.assets-gap.com/webcontent/0016/70...,Polka Dot Bi-Stretch Sheath Dress,https://bananarepublic.gap.com//browse/product...,139.0,Banana Republic,A classic sheath dress made with our Bi-Stretc...
8,https://www4.assets-gap.com/webcontent/0016/63...,Soft Ponte Twist-Front Dress,https://bananarepublic.gap.com//browse/product...,98.5,Banana Republic,Delicate draping across the bodice adds a Grec...
9,https://www4.assets-gap.com/webcontent/0016/88...,Stripe Ponte Tank Dress,https://bananarepublic.gap.com//browse/product...,119.0,Banana Republic,"A column-style tank dress, made in a soft, kni..."


In [67]:
cat = ['https://bananarepublic.gap.com/browse/category.do?cid=5013&mlink=5001,,flyout_women_apparel_suits&clink=15682852',
      'https://bananarepublic.gap.com/browse/category.do?cid=5032&mlink=5001,,flyout_women_apparel_sweaters&clink=15682852',
      'https://bananarepublic.gap.com/browse/category.do?cid=5037&mlink=5001,,flyout_women_apparel_blouses&clink=15682852',
      'https://bananarepublic.gap.com/browse/category.do?cid=87056&mlink=5001,,flyout_women_apparel_blazers&clink=15682852',
      'https://bananarepublic.gap.com/browse/category.do?cid=99915&mlink=5001,,flyout_women_apparel_coats_jackets&clink=15682852',
      'https://bananarepublic.gap.com/browse/category.do?cid=67595&mlink=5001,,flyout_women_apparel_pants&clink=15682852',
      'https://bananarepublic.gap.com/browse/category.do?cid=94275&mlink=5001,,flyout_women_apparel_shorts&clink=15682852',
      'https://bananarepublic.gap.com/browse/category.do?cid=5022&mlink=5001,,flyout_women_apparel_skirts&clink=15682852']

In [68]:
prod_pages = []
prod_names = []
prod_prices = []
prod_dp = []
prod_images = []
prod_desc=[]
for cat_link in cat:
    driver.get(cat_link)
    for i in range(100):
        driver.execute_script("window.scrollTo("+str(i*500)+", "+str((i+1)*500)+");")
        main_page = driver.page_source
        main_soup = BeautifulSoup(main_page, "lxml")
        prod_pages,prod_names,prod_prices,prod_images,prod_desc = scrape(prod_pages,prod_names,prod_prices,prod_images,prod_desc,main_soup)

In [69]:
len(prod_pages),len(prod_images),len(prod_names),len(prod_prices)

(114778, 114778, 114778, 114778)

In [70]:
d2 = { 'image_link': prod_images,
         'product_name': prod_names, 
        'link': prod_pages, 
         'price': prod_prices}
#,
#    'description':prod_desc}
    
df2 = pd.DataFrame(d2)
df2['domain'] = 'Banana Republic'

In [71]:
df2 = df2.dropna().drop_duplicates()

In [72]:
len(df2)

1167

In [93]:
#img = []
#prod_desc = []
for url in df2.link[571:]:
    driver.get(url)
    desc = driver.page_source
    desc_soup = BeautifulSoup(desc, "lxml")
    d = desc_soup.find('li',{'class':'product-information-item__list-item'})
    if d:
        description = d.get_text()
        prod_desc.append(description)
        alt_img = desc_soup.find_all('a',{'class':'hover-zoom hover-zoom-in'})
        if len(alt_img) > 1:
            alt_img = alt_img[1]
        else:
            alt_img = alt_img[0]
        img.append('https://bananarepublic.gap.com'+alt_img.img['src'])
    else:
        prod_desc.append(None)
        img.append(None)

In [94]:
len(img),len(prod_desc)

(1167, 1167)

In [96]:
df2.head()

,image_link,product_name,link,price,domain
0,https://www4.assets-gap.com/webcontent/0016/83...,Long and Lean-Fit Linen-Cotton Blazer,https://bananarepublic.gap.com//browse/product...,$179.00,Banana Republic
1,https://www2.assets-gap.com/webcontent/0016/83...,Avery Straight-Fit Linen-Cotton Pant,https://bananarepublic.gap.com//browse/product...,$98.50,Banana Republic
2,https://www1.assets-gap.com/webcontent/0016/87...,Long and Lean-Fit Linen-Cotton Blazer,https://bananarepublic.gap.com//browse/product...,$179.00,Banana Republic
3,https://www3.assets-gap.com/webcontent/0016/87...,Avery Straight-Fit Linen-Cotton Pant,https://bananarepublic.gap.com//browse/product...,$98.50,Banana Republic
4,https://www4.assets-gap.com/webcontent/0016/85...,Long and Lean-Fit Linen-Cotton Blazer,https://bananarepublic.gap.com//browse/product...,$179.00,Banana Republic


In [97]:
df2['description'] = prod_desc
#df2['']

In [99]:
df2['price']=df2.price.map(lambda x: float(x[1:]))
df2

,image_link,product_name,link,price,domain,description
0,https://www4.assets-gap.com/webcontent/0016/83...,Long and Lean-Fit Linen-Cotton Blazer,https://bananarepublic.gap.com//browse/product...,179.0,Banana Republic,Made with a soft and summer-ready blend of lin...
1,https://www2.assets-gap.com/webcontent/0016/83...,Avery Straight-Fit Linen-Cotton Pant,https://bananarepublic.gap.com//browse/product...,98.5,Banana Republic,The Avery has a trouser fit through the hip an...
2,https://www1.assets-gap.com/webcontent/0016/87...,Long and Lean-Fit Linen-Cotton Blazer,https://bananarepublic.gap.com//browse/product...,179.0,Banana Republic,Made with a soft and summer-ready blend of lin...
3,https://www3.assets-gap.com/webcontent/0016/87...,Avery Straight-Fit Linen-Cotton Pant,https://bananarepublic.gap.com//browse/product...,98.5,Banana Republic,The Avery has a trouser fit through the hip an...
4,https://www4.assets-gap.com/webcontent/0016/85...,Long and Lean-Fit Linen-Cotton Blazer,https://bananarepublic.gap.com//browse/product...,179.0,Banana Republic,Made with a soft and summer-ready blend of lin...
5,https://www4.assets-gap.com/webcontent/0016/70...,Avery Straight-Fit Linen-Cotton Pant,https://bananarepublic.gap.com//browse/product...,98.5,Banana Republic,The Avery has a trouser fit through the hip an...
6,https://www2.assets-gap.com/webcontent/0016/80...,Classic-Fit Linen-Cotton Blazer,https://bananarepublic.gap.com//browse/product...,179.0,Banana Republic,Made with a soft and summer-ready blend of lin...
7,https://www1.assets-gap.com/webcontent/0016/67...,Avery Straight-Fit Linen-Cotton Pant,https://bananarepublic.gap.com//browse/product...,98.5,Banana Republic,The Avery has a trouser fit through the hip an...
8,https://www2.assets-gap.com/webcontent/0016/84...,Classic-Fit Linen-Cotton Blazer,https://bananarepublic.gap.com//browse/product...,179.0,Banana Republic,Made with a soft and summer-ready blend of lin...
257,https://www1.assets-gap.com/webcontent/0016/70...,Avery Straight-Fit Linen-Cotton Pant,https://bananarepublic.gap.com//browse/product...,98.5,Banana Republic,The Avery has a trouser fit through the hip an...


In [106]:
banana_repbulic = pd.concat([df,df2],axis=0).dropna().reset_index()[['image_link','product_name','link','price','description','domain']]

In [ ]:
banana_repbulic.csv('banana_republic.csv')